In [1]:
# 패키지를 설치 안 했다면 아래 명령어를 주석을 해제하고 실행시키세요.
!pip install -r ../../requirements.txt

  Using cached pandas-2.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached scikit_learn-1.7.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached matplotlib-3.10.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached notebook-7.4.3-py3-none-any.whl.metadata (10 kB)
  Using cached catboost-1.2.8-cp312-cp312-macosx_11_0_universal2.whl.metadata (1.4 kB)
  Using cached plotly-6.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached scipy-1.15.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.

In [2]:
# 실거주 인구 데이터 다운로드
import os
import zipfile
import requests

# 경로 설정
base_dir = "../../Data"
localpeople_dir = os.path.join(base_dir, "LocalPeople")
zip_url = (
    "https://huggingface.co/datasets/uhjin1130/LocalPeople/resolve/main/LocalPeople.zip"
)
zip_path = os.path.join(localpeople_dir, "LocalPeople.zip")

# LocalPeople 폴더가 없으면 생성 및 다운로드
if not os.path.exists(localpeople_dir):
    print("📁 LocalPeople 폴더가 없습니다. 다운로드 및 압축 해제 시작...")

    os.makedirs(localpeople_dir, exist_ok=True)

    # 다운로드
    print("🔽 LocalPeople.zip 다운로드 중...")
    response = requests.get(zip_url)
    with open(zip_path, "wb") as f:
        f.write(response.content)
    print("✅ 다운로드 완료!")

    # 압축 해제 (해당 폴더 안으로)
    print("📦 압축 해제 중...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(localpeople_dir)
    print("✅ 압축 해제 완료!")

    # 압축 파일 삭제
    os.remove(zip_path)
    print("🧹 ZIP 파일 삭제 완료!")

else:
    print("✅ LocalPeople 폴더가 이미 존재합니다. 다운로드 생략.")

📁 LocalPeople 폴더가 없습니다. 다운로드 및 압축 해제 시작...
🔽 LocalPeople.zip 다운로드 중...
✅ 다운로드 완료!
📦 압축 해제 중...
✅ 압축 해제 완료!
🧹 ZIP 파일 삭제 완료!


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import warnings
import gc
import joblib
from datetime import datetime

warnings.filterwarnings("ignore")

In [4]:
def load_localpeople_quarterly_fixed(year, quarter, people_dir):
    """수정된 분기별 LocalPeople 데이터 로드"""

    quarter_months = {1: [1, 2, 3], 2: [4, 5, 6], 3: [7, 8, 9], 4: [10, 11, 12]}

    months = quarter_months[quarter]
    print(f"    {year}년 {quarter}분기 인구 데이터 로드 중...")

    monthly_agg_list = []

    for month in months:
        month_str = f"{year}{month:02d}"
        file_path = os.path.join(people_dir, f"LOCAL_PEOPLE_DONG_{month_str}.csv")

        if not os.path.exists(file_path):
            continue

        try:
            # chunk 단위로 읽기
            chunk_list = []
            chunksize = 50000

            # 🔥 CSV 읽기 문제 해결: dtype 강제 지정 + BOM 처리
            expected_cols = [
                "기준일ID",
                "시간대구분",
                "행정동코드",
                "총생활인구수",
                "남자0세부터9세생활인구수",
                "남자10세부터14세생활인구수",
                "남자15세부터19세생활인구수",
                "남자20세부터24세생활인구수",
                "남자25세부터29세생활인구수",
                "남자30세부터34세생활인구수",
                "남자35세부터39세생활인구수",
                "남자40세부터44세생활인구수",
                "남자45세부터49세생활인구수",
                "남자50세부터54세생활인구수",
                "남자55세부터59세생활인구수",
                "남자60세부터64세생활인구수",
                "남자65세부터69세생활인구수",
                "남자70세이상생활인구수",
                "여자0세부터9세생활인구수",
                "여자10세부터14세생활인구수",
                "여자15세부터19세생활인구수",
                "여자20세부터24세생활인구수",
                "여자25세부터29세생활인구수",
                "여자30세부터34세생활인구수",
                "여자35세부터39세생활인구수",
                "여자40세부터44세생활인구수",
                "여자45세부터49세생활인구수",
                "여자50세부터54세생활인구수",
                "여자55세부터59세생활인구수",
                "여자60세부터64세생활인구수",
                "여자65세부터69세생활인구수",
                "여자70세이상생활인구수",
            ]

            try:
                df_iter = pd.read_csv(
                    file_path,
                    encoding="utf-8",
                    chunksize=chunksize,
                    dtype={
                        "기준일ID": str,
                        "시간대구분": str,
                        "행정동코드": str,
                    },  # 핵심: str로 강제
                    usecols=expected_cols,  # 정확한 컬럼만 선택
                    header=0,  # 첫 번째 행이 헤더임을 명시
                )
            except:
                df_iter = pd.read_csv(
                    file_path,
                    encoding="cp949",
                    chunksize=chunksize,
                    dtype={"기준일ID": str, "시간대구분": str, "행정동코드": str},
                    usecols=expected_cols,
                    header=0,
                )

            for chunk in df_iter:
                chunk["행정동코드"] = chunk["행정동코드"].astype(str).str.zfill(8)

                # 필요한 컬럼만 선택
                numeric_cols = ["총생활인구수"] + [
                    col
                    for col in chunk.columns
                    if ("남자" in col or "여자" in col) and "생활인구수" in col
                ]
                keep_cols = ["행정동코드"] + numeric_cols
                chunk = chunk[keep_cols]

                # 행정동별 집계
                chunk_agg = (
                    chunk.groupby("행정동코드").sum(numeric_only=True).reset_index()
                )
                chunk_list.append(chunk_agg)

            # 월별 데이터 합치기
            if chunk_list:
                month_df = pd.concat(chunk_list, ignore_index=True)
                month_agg = (
                    month_df.groupby("행정동코드").sum(numeric_only=True).reset_index()
                )
                monthly_agg_list.append(month_agg)
                print(f"      {month_str} 완료 ({len(month_agg)}개 행정동)")

            del chunk_list
            gc.collect()

        except Exception as e:
            print(f"      {month_str} 로드 실패: {e}")
            continue

    if not monthly_agg_list:
        return None

    # 분기별 평균 계산
    quarter_df = monthly_agg_list[0]
    for month_df in monthly_agg_list[1:]:
        quarter_df = pd.merge(
            quarter_df,
            month_df,
            on="행정동코드",
            how="outer",
            suffixes=("", "_temp"),
        )

        # 평균 계산
        for col in month_df.columns:
            if col != "행정동코드":
                if f"{col}_temp" in quarter_df.columns:
                    quarter_df[col] = quarter_df[[col, f"{col}_temp"]].sum(
                        axis=1, skipna=True
                    )
                    quarter_df = quarter_df.drop(columns=[f"{col}_temp"])

    # 이제 행정동코드는 이미 올바른 컬럼에 있음

    print(
        f"    {year}년 {quarter}분기 인구 데이터 집계 완료: {len(quarter_df)}개 행정동"
    )
    return quarter_df

In [5]:
def load_trading_area_data(year, biz_dir):
    """Trading Area 데이터 로드"""
    file_path = os.path.join(biz_dir, f"Trading_Area_{year}.csv")

    if not os.path.exists(file_path):
        return None

    try:
        try:
            df = pd.read_csv(file_path, encoding="utf-8")
        except:
            df = pd.read_csv(file_path, encoding="cp949")

        df = df.rename(columns={"행정동_코드": "행정동코드"})
        df["행정동코드"] = df["행정동코드"].astype(str).str.zfill(8)

        print(f"  Trading Area {year} 로드 완료: {len(df)}행")
        return df

    except Exception as e:
        print(f"  Trading Area {year} 로드 실패: {e}")
        return None

In [6]:
def create_master_dataset_fixed(years, people_dir, biz_dir):
    """수정된 마스터 데이터셋 생성"""
    print("=== 수정된 마스터 데이터셋 생성 시작 ===")

    all_data = []

    for year in years:
        print(f"\n📅 {year}년 데이터 처리 중...")

        # Trading Area 데이터 로드
        biz_df = load_trading_area_data(year, biz_dir)
        if biz_df is None:
            continue

        # 각 분기별로 처리
        for quarter in [1, 2, 3, 4]:
            print(f"  🔄 {year}년 {quarter}분기 처리 중...")

            # 해당 분기 Trading Area 데이터 필터링
            quarter_code = int(f"{year}{quarter}")
            biz_quarter = biz_df[biz_df["기준_년분기_코드"] == quarter_code].copy()

            if len(biz_quarter) == 0:
                print(f"    ❌ {year}년 {quarter}분기 매출 데이터 없음")
                continue

            # LocalPeople 데이터 로드 (수정된 버전)
            people_quarter = load_localpeople_quarterly_fixed(year, quarter, people_dir)

            if people_quarter is None:
                print(f"    ❌ {year}년 {quarter}분기 인구 데이터 없음")
                continue

            # 데이터 병합
            merged_data = pd.merge(
                biz_quarter, people_quarter, on="행정동코드", how="left"
            )

            # 병합 결과 확인 (첫 번째 인구 컬럼으로 매칭률 확인)
            people_cols = [col for col in people_quarter.columns if col != "행정동코드"]
            if people_cols:
                people_match_rate = (
                    (~merged_data[people_cols[0]].isna()).sum() / len(merged_data) * 100
                )
            else:
                people_match_rate = 0

            print(
                f"    ✅ 병합 완료: 매출 {len(biz_quarter)} + 인구 {len(people_quarter)} → {len(merged_data)}행"
            )
            print(f"    📊 인구 데이터 매칭률: {people_match_rate:.1f}%")

            all_data.append(merged_data)

            # 메모리 정리
            del biz_quarter, people_quarter, merged_data
            gc.collect()

    if not all_data:
        raise ValueError("생성된 데이터가 없습니다!")

    # 전체 데이터 결합
    print("\n🔗 전체 데이터 결합 중...")
    final_df = pd.concat(all_data, ignore_index=True)
    print(f"✅ 마스터 데이터셋 생성 완료: {final_df.shape}")

    return final_df

In [7]:
def preprocess_data_fixed(df):
    """수정된 데이터 전처리"""
    print("\n=== 데이터 전처리 시작 ===")
    print(f"원본 데이터 크기: {df.shape}")

    # 타겟 변수 확인
    if "당월_매출_금액" not in df.columns:
        raise ValueError("타겟 변수 '당월_매출_금액'이 없습니다!")

    # 불필요한 컬럼 제거 + 데이터 리키지 방지
    drop_cols = ["행정동_코드_명", "서비스_업종_코드_명"]

    # 🔥 매출 관련 특성 제거 (데이터 리키지 방지)
    sales_cols = [
        col
        for col in df.columns
        if any(
            keyword in col
            for keyword in [
                "매출_금액",
                "매출_건수",
                "월요일_",
                "화요일_",
                "수요일_",
                "목요일_",
                "금요일_",
                "토요일_",
                "일요일_",
                "주중_",
                "주말_",
                "시간대_",
                "남성_매출",
                "여성_매출",
                "연령대_",
            ]
        )
    ]

    # 타겟 변수는 유지
    sales_cols = [col for col in sales_cols if col != "당월_매출_금액"]

    print(f"데이터 리키지 방지를 위해 제거할 매출 관련 컬럼: {len(sales_cols)}개")
    print(f"제거 컬럼 예시: {sales_cols[:5]}")

    drop_cols.extend(sales_cols)
    df = df.drop(columns=[col for col in drop_cols if col in df.columns])

    # 결측값이 너무 많은 컬럼 제거 (70% 이상)
    null_ratio = df.isnull().mean()
    high_null_cols = null_ratio[null_ratio > 0.7].index.tolist()
    print(f"결측률 70% 이상 컬럼 제거: {len(high_null_cols)}개")
    if high_null_cols:
        df = df.drop(columns=high_null_cols)

    # 범주형 변수 인코딩
    if "서비스_업종_코드" in df.columns:
        le = LabelEncoder()
        df["서비스_업종_코드_encoded"] = le.fit_transform(
            df["서비스_업종_코드"].astype(str)
        )
        df = df.drop(columns=["서비스_업종_코드"])

    # LocalPeople 관련 컬럼 확인
    people_cols = [
        col
        for col in df.columns
        if any(keyword in col for keyword in ["생활인구수", "남자", "여자"])
    ]
    print(f"📊 LocalPeople 관련 컬럼: {len(people_cols)}개")

    if len(people_cols) == 0:
        print("⚠️ 경고: LocalPeople 데이터가 포함되지 않았습니다!")
    else:
        print("✅ LocalPeople 데이터 포함 확인됨")
        # 몇 개 컬럼명 출력
        sample_cols = people_cols[:5]
        print(f"   예시: {sample_cols}")

    print(f"전처리 후 데이터 크기: {df.shape}")
    return df

In [8]:
def train_and_evaluate_model_fixed(X_train, y_train, X_test, y_test):
    """수정된 모델 학습 및 평가"""
    print("\n=== 모델 학습 및 평가 ===")
    print(f"훈련 데이터: {X_train.shape}, 테스트 데이터: {X_test.shape}")

    # 타겟 변수 로그 변환
    y_train_log = np.log1p(y_train.clip(lower=0))

    # RandomForest 모델
    model = RandomForestRegressor(
        n_estimators=100, max_depth=15, random_state=42, n_jobs=-1, oob_score=True
    )

    print("🚀 모델 학습 시작...")
    model.fit(X_train, y_train_log)
    print("✅ 모델 학습 완료!")

    # 예측
    print("🔮 예측 중...")
    y_pred_log = model.predict(X_test)
    y_pred = np.expm1(y_pred_log)
    y_pred = np.clip(y_pred, 0, None)  # 음수 제거

    # 성능 평가
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)

    print("\n" + "=" * 60)
    print("🎯 수정된 베이스라인 모델 성능 평가 결과")
    print("=" * 60)
    print(f"MSE:  {mse:,.0f}")
    print(f"RMSE: {rmse:,.0f} 원")
    print(f"MAE:  {mae:,.0f} 원")
    print(f"테스트 데이터 평균 매출: {y_test.mean():,.0f} 원")
    print(f"Out-of-Bag Score: {model.oob_score_:.4f}")

    # 상위 20개 특성 중요도 출력
    feature_importance = pd.DataFrame(
        {"feature": X_train.columns, "importance": model.feature_importances_}
    ).sort_values("importance", ascending=False)

    print(f"\n--- 상위 20개 특성 중요도 ---")
    for i, (_, row) in enumerate(feature_importance.head(20).iterrows(), 1):
        is_people = any(
            keyword in row["feature"] for keyword in ["생활인구수", "남자", "여자"]
        )
        marker = "🏠" if is_people else "🏪"
        print(f"{i:2d}. {marker} {row['feature'][:50]}: {row['importance']:.4f}")

    # LocalPeople 데이터 기여도
    people_features = feature_importance[
        feature_importance["feature"].str.contains("생활인구수|남자|여자", regex=True)
    ]
    people_importance = people_features["importance"].sum()

    print(
        f"\n📊 LocalPeople 데이터 총 기여도: {people_importance:.4f} ({people_importance*100:.1f}%)"
    )

    if people_importance < 0.05:
        print("⚠️ LocalPeople 데이터 기여도가 매우 낮습니다.")
    elif people_importance < 0.15:
        print("🔶 LocalPeople 데이터 기여도가 낮습니다.")
    else:
        print("✅ LocalPeople 데이터가 유의미하게 기여하고 있습니다.")

    return (
        model,
        {"mse": mse, "rmse": rmse, "mae": mae},
        feature_importance,
        people_importance,
    )

In [9]:
def save_model_and_metrics(
    model, metrics, feature_importance_df, people_importance, config
):
    """모델과 평가지표 저장"""

    # 결과 저장 디렉토리 생성 (현재 스크립트 디렉토리 기준)
    script_dir = os.getcwd()
    results_dir = os.path.join(script_dir, "results")
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        print(f"📁 결과 디렉토리 생성: {results_dir}")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 1. 모델 저장
    model_filename = os.path.join(results_dir, f"localpeople_model_{timestamp}.joblib")
    joblib.dump(model, model_filename)
    print(f"💾 모델 저장 완료: {model_filename}")

    # 2. 평가지표 CSV 저장
    metrics_data = {
        "실행시간": [timestamp],
        "MSE": [metrics["mse"]],
        "RMSE": [metrics["rmse"]],
        "MAE": [metrics["mae"]],
        "LocalPeople_기여도": [people_importance],
        "LocalPeople_기여도_퍼센트": [people_importance * 100],
        "OOB_Score": [model.oob_score_],
        "테스트_년도": [config["test_year"]],
        "훈련_데이터_크기": [config["train_size"]],
        "테스트_데이터_크기": [config["test_size"]],
        "특성_개수": [config["n_features"]],
    }

    metrics_df = pd.DataFrame(metrics_data)
    metrics_filename = os.path.join(results_dir, f"localpeople_metrics_{timestamp}.csv")
    metrics_df.to_csv(metrics_filename, index=False, encoding="utf-8-sig")
    print(f"📊 평가지표 CSV 저장 완료: {metrics_filename}")

    # 3. 특성 중요도 저장
    importance_filename = os.path.join(
        results_dir, f"localpeople_importance_{timestamp}.csv"
    )
    feature_importance_df.to_csv(importance_filename, index=False, encoding="utf-8-sig")
    print(f"🔍 특성 중요도 CSV 저장 완료: {importance_filename}")

    # 4. 실행 정보 요약 저장
    summary_data = {
        "항목": [
            "실행시간",
            "MSE",
            "RMSE (원)",
            "MAE (원)",
            "LocalPeople 기여도 (%)",
            "OOB Score",
            "테스트 년도",
            "훈련 데이터 크기",
            "테스트 데이터 크기",
            "특성 개수",
        ],
        "값": [
            timestamp,
            f"{metrics['mse']:,.0f}",
            f"{metrics['rmse']:,.0f}",
            f"{metrics['mae']:,.0f}",
            f"{people_importance*100:.1f}%",
            f"{model.oob_score_:.4f}",
            config["test_year"],
            f"{config['train_size']:,}",
            f"{config['test_size']:,}",
            config["n_features"],
        ],
    }

    summary_df = pd.DataFrame(summary_data)
    summary_filename = os.path.join(results_dir, f"localpeople_summary_{timestamp}.csv")
    summary_df.to_csv(summary_filename, index=False, encoding="utf-8-sig")
    print(f"📋 모델 요약 CSV 저장 완료: {summary_filename}")

    print(f"\n🎉 모든 결과가 '{results_dir}' 폴더에 저장되었습니다!")
    return {
        "model_file": model_filename,
        "metrics_file": metrics_filename,
        "importance_file": importance_filename,
        "summary_file": summary_filename,
    }

In [ ]:
print("🚀 수정된 상권 매출 예측 베이스라인 모델")
print("=" * 50)

# 설정
YEARS = range(2019, 2025)
PEOPLE_DIR = "../../Data/LocalPeople"
BIZ_DIR = "../../Data/Trading_Area"
TEST_YEAR = 2024

# 1. 데이터 로드 및 병합 (수정된 버전)
master_df = create_master_dataset_fixed(YEARS, PEOPLE_DIR, BIZ_DIR)

# 2. 데이터 전처리
processed_df = preprocess_data_fixed(master_df)

🚀 수정된 상권 매출 예측 베이스라인 모델
=== 수정된 마스터 데이터셋 생성 시작 ===

📅 2019년 데이터 처리 중...
  Trading Area 2019 로드 완료: 65666행
  🔄 2019년 1분기 처리 중...
    2019년 1분기 인구 데이터 로드 중...
      201901 완료 (424개 행정동)
      201902 완료 (424개 행정동)
      201903 완료 (424개 행정동)
    2019년 1분기 인구 데이터 집계 완료: 424개 행정동
    ✅ 병합 완료: 매출 16273 + 인구 424 → 16273행
    📊 인구 데이터 매칭률: 98.5%
  🔄 2019년 2분기 처리 중...
    2019년 2분기 인구 데이터 로드 중...
      201904 완료 (424개 행정동)
      201905 완료 (424개 행정동)
      201906 완료 (424개 행정동)
    2019년 2분기 인구 데이터 집계 완료: 424개 행정동
    ✅ 병합 완료: 매출 16418 + 인구 424 → 16418행
    📊 인구 데이터 매칭률: 98.5%
  🔄 2019년 3분기 처리 중...
    2019년 3분기 인구 데이터 로드 중...
      201907 완료 (424개 행정동)
      201908 완료 (424개 행정동)
      201909 완료 (424개 행정동)
    2019년 3분기 인구 데이터 집계 완료: 424개 행정동
    ✅ 병합 완료: 매출 16456 + 인구 424 → 16456행
    📊 인구 데이터 매칭률: 98.5%
  🔄 2019년 4분기 처리 중...
    2019년 4분기 인구 데이터 로드 중...
      201910 완료 (424개 행정동)
      201911 완료 (424개 행정동)
      201912 완료 (424개 행정동)
    2019년 4분기 인구 데이터 집계 완료: 424개 행정동
    ✅ 병합 완료: 매출 16519 + 인구

In [12]:
# 3. 데이터 분할
print(f"\n=== 데이터 분할 ({TEST_YEAR}년 테스트) ===")
train_mask = processed_df["기준_년분기_코드"] < (TEST_YEAR * 10 + 1)
test_mask = processed_df["기준_년분기_코드"] >= (TEST_YEAR * 10 + 1)

train_df = processed_df[train_mask].copy()
test_df = processed_df[test_mask].copy()

print(f"훈련 데이터: {len(train_df)}행")
print(f"테스트 데이터: {len(test_df)}행")


=== 데이터 분할 (2024년 테스트) ===
훈련 데이터: 340321행
테스트 데이터: 67900행


In [13]:
# 특성과 타겟 분리
target_col = "당월_매출_금액"
feature_cols = [col for col in train_df.columns if col != target_col]

X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]

# 결측값 처리
print("결측값 처리 중...")
imputer = SimpleImputer(strategy="mean")
X_train_filled = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_filled = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

결측값 처리 중...


In [14]:
# 4. 모델 학습 및 평가
model, metrics, feature_importance, people_importance = train_and_evaluate_model_fixed(
    X_train_filled, y_train, X_test_filled, y_test
)

print("\n🎉 수정된 베이스라인 모델 학습 완료!")
print("=" * 50)


=== 모델 학습 및 평가 ===
훈련 데이터: (340321, 32), 테스트 데이터: (67900, 32)
🚀 모델 학습 시작...
✅ 모델 학습 완료!
🔮 예측 중...

🎯 수정된 베이스라인 모델 성능 평가 결과
MSE:  89,297,344,592,772,399,104
RMSE: 9,449,727,223 원
MAE:  826,261,234 원
테스트 데이터 평균 매출: 1,532,612,027 원
Out-of-Bag Score: 0.6895

--- 상위 20개 특성 중요도 ---
 1. 🏪 서비스_업종_코드_encoded: 0.5833
 2. 🏪 행정동코드: 0.0478
 3. 🏠 여자25세부터29세생활인구수: 0.0404
 4. 🏠 남자30세부터34세생활인구수: 0.0345
 5. 🏠 남자35세부터39세생활인구수: 0.0231
 6. 🏠 남자50세부터54세생활인구수: 0.0164
 7. 🏠 여자30세부터34세생활인구수: 0.0156
 8. 🏠 여자10세부터14세생활인구수: 0.0155
 9. 🏠 여자20세부터24세생활인구수: 0.0138
10. 🏠 남자0세부터9세생활인구수: 0.0137
11. 🏠 남자25세부터29세생활인구수: 0.0132
12. 🏠 남자45세부터49세생활인구수: 0.0132
13. 🏠 남자40세부터44세생활인구수: 0.0131
14. 🏠 여자0세부터9세생활인구수: 0.0125
15. 🏠 여자15세부터19세생활인구수: 0.0116
16. 🏠 여자70세이상생활인구수: 0.0116
17. 🏠 남자15세부터19세생활인구수: 0.0113
18. 🏠 남자10세부터14세생활인구수: 0.0106
19. 🏪 기준_년분기_코드: 0.0095
20. 🏠 남자20세부터24세생활인구수: 0.0093

📊 LocalPeople 데이터 총 기여도: 0.3595 (35.9%)
✅ LocalPeople 데이터가 유의미하게 기여하고 있습니다.

🎉 수정된 베이스라인 모델 학습 완료!


In [15]:
# 5. 모델 및 결과 저장
config = {
    "test_year": TEST_YEAR,
    "train_size": len(train_df),
    "test_size": len(test_df),
    "n_features": len(feature_cols),
}
result = save_model_and_metrics(
    model, metrics, feature_importance, people_importance, config
)

print("\n🎉 모든 결과가 저장되었습니다!")
print("=" * 50)

💾 모델 저장 완료: /Users/rubidium/Project/BDP/test/BigDataProgramming_TeamI/Code/Analyze_with_LocalPeople/results/localpeople_model_20250619_212155.joblib
📊 평가지표 CSV 저장 완료: /Users/rubidium/Project/BDP/test/BigDataProgramming_TeamI/Code/Analyze_with_LocalPeople/results/localpeople_metrics_20250619_212155.csv
🔍 특성 중요도 CSV 저장 완료: /Users/rubidium/Project/BDP/test/BigDataProgramming_TeamI/Code/Analyze_with_LocalPeople/results/localpeople_importance_20250619_212155.csv
📋 모델 요약 CSV 저장 완료: /Users/rubidium/Project/BDP/test/BigDataProgramming_TeamI/Code/Analyze_with_LocalPeople/results/localpeople_summary_20250619_212155.csv

🎉 모든 결과가 '/Users/rubidium/Project/BDP/test/BigDataProgramming_TeamI/Code/Analyze_with_LocalPeople/results' 폴더에 저장되었습니다!

🎉 모든 결과가 저장되었습니다!
